Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# AutoML: Energy Demand Forecasting
Time series forecasting is the task of predicting future values in a time-ordered sequence of observations. It is a common problem and has applications in many industries. For example, retail companies need to forecast future product sales so they can effectively organize their supply chains to meet demand. Similarly, package delivery companies need to estimate the demand for their services so they can plan workforce requirements and delivery routes ahead of time. In many cases, the financial risks of inaccurate forecasts can be significant. Therefore, forecasting is often a business critical activity.

This sample shows how time series forecasting can be performed through AutoML package withing AzureML Services.

Make sure you have executed the [00.configuration](00.configuration.ipynb) before running this notebook.

In this notebook you would see
1. Creating an Experiment in an existing Workspace
2. Instantiating AutoMLConfig
3. Training the Model using local compute
4. Exploring the results
5. Testing the fitted model


## Create Experiment

As part of the setup you have already created a <b>Workspace</b>. For AutoML you would need to create an <b>Experiment</b>. An <b>Experiment</b> is a named object in a <b>Workspace</b>, which is used to run experiments.

In [1]:
import azureml.core
import pandas as pd
import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
import time
import logging
import os
from sklearn import datasets
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import random
import numpy as np
from scipy import stats

ModuleNotFoundError: No module named 'azureml.core.experiment'

In [2]:
ws = Workspace.from_config()

# choose a name for the run history container in the workspace
history_name = 'automl-energydemandforecasting'

# project folder
project_folder = './sample_projects/automl-energydemandforecasting'

import os
from azureml.core.project import Project

project = Project.attach(ws,
                         history_name,
                         project_folder)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project.project_directory
output['Run History Name'] = project.history.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

Found the config file in: C:\Users\lazzeri\Desktop\MachineLearningNotebooks-master\automl\aml_config\config.json


,
SDK version,0.1.4
Subscription ID,ff18d7a8-962a-406c-858f-49acd23d6c01
Workspace Name,myws
Resource Group,myrg
Location,eastus2
Project Directory,C:\Users\lazzeri\Desktop\MachineLearningNotebooks-master\automl\sample_projects\automl-energydemandforecasting
Run History Name,automl-energydemandforecasting


### Update Conda Dependency file to have AutoML SDK

Currently AutoML SDK is not installed with Azure ML SDK by default, Due to this we update the conda dependency file to add a dependency on AutoML SDK.

In [3]:
%%writefile $project_folder/aml_config/conda_dependencies.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for managed runs. These include runs against
# the localdocker, remotedocker, and cluster compute targets.

# Note that this file is NOT used to automatically manage dependencies for the
# local compute target. To provision these dependencies locally, run:
# conda env update --file conda_dependencies.yml

# Details about the Conda environment file format:
# https://conda.io/docs/using/envs.html#create-environment-file-by-hand

# For managing Spark packages and configuration, see spark_dependencies.yml.

# Version of this configuration file's structure and semantics in AzureML.
# This directive is stored in a comment to preserve the Conda file structure.
# [AzureMlVersion] = 2

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML Workbench only supports 3.5.2 and later.
  - python=3.6.2
  # Required by azureml-requirements, installed separately through Conda to
  # get a prebuilt version and not require build tools for the install.
  - psutil=5.4.5
  - numpy

  - pip:
    # Required packages for AzureML execution, history, and data preparation.
    - --index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1
    - --extra-index-url https://pypi.python.org/simple
    - azureml-sdk
    - azureml-train-automl

Overwriting ./sample_projects/automl-energydemandforecasting/aml_config/conda_dependencies.yml


## Create Get Data File
For remote executions you should author a get_data.py file containing a get_data() function. This file should be in the root directory of the project. You can encapsulate code to read data either from a blob storage or local disk in this file.

The *get_data()* function returns a [dictionary](README.md#getdata).

In [4]:
%%writefile $project_folder/get_data.py

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def get_data():
    
    demand = pd.read_csv("https://antaignitedata.blob.core.windows.net/antaignitedata/nyc_demand.csv", parse_dates=['timeStamp'])
    weather = pd.read_csv("https://antaignitedata.blob.core.windows.net/antaignitedata/nyc_weather.csv", parse_dates=['timeStamp'])
    df = pd.merge(demand, weather, on=['timeStamp'], how='outer')
     
    return { "X" : df }

Writing ./sample_projects/automl-energydemandforecasting/get_data.py


### View data

You can execute the *get_data()* function locally to view the *energy* data

In [5]:
%run  $project_folder/get_data.py
data_dict = get_data()
df = data_dict["X"]
df.head()

,timeStamp,demand,precip,temp
0,2012-01-01 00:00:00,4937.5,0.0,46.13
1,2012-01-01 01:00:00,4752.1,0.0,45.89
2,2012-01-01 02:00:00,4542.6,0.0,45.04
3,2012-01-01 03:00:00,4357.7,0.0,45.03
4,2012-01-01 04:00:00,4275.5,0.0,42.61


## Fill gaps in the time series
Some periods in the time series are missing. This occurs if the period was missing in both the original demand and weather datasets. To identify these gaps, first we create an index of time periods that we would expect to be in the time series. There should be one record for every hour between the minimum and maximum datetimes in our dataset.

In [6]:
min_time = min(df['timeStamp'])
min_time

Timestamp('2012-01-01 00:00:00')

In [7]:
max_time = max(df['timeStamp'])
max_time

Timestamp('2017-08-12 06:00:00')

In [8]:
dt_idx = pd.date_range(min_time, max_time, freq='H')
dt_idx

DatetimeIndex(['2012-01-01 00:00:00', '2012-01-01 01:00:00',
               '2012-01-01 02:00:00', '2012-01-01 03:00:00',
               '2012-01-01 04:00:00', '2012-01-01 05:00:00',
               '2012-01-01 06:00:00', '2012-01-01 07:00:00',
               '2012-01-01 08:00:00', '2012-01-01 09:00:00',
               ...
               '2017-08-11 21:00:00', '2017-08-11 22:00:00',
               '2017-08-11 23:00:00', '2017-08-12 00:00:00',
               '2017-08-12 01:00:00', '2017-08-12 02:00:00',
               '2017-08-12 03:00:00', '2017-08-12 04:00:00',
               '2017-08-12 05:00:00', '2017-08-12 06:00:00'],
              dtype='datetime64[ns]', length=49207, freq='H')

Now we index the dataframe according to this datetime index to insert missing records into the time series:

In [9]:
df.index = df['timeStamp']
df = df.reindex(dt_idx)

df[df.isnull().all(axis=1)]

,timeStamp,demand,precip,temp
2016-03-13 02:00:00,NaT,NaN,NaN,NaN
2017-03-12 02:00:00,NaT,NaN,NaN,NaN


Now that there are no missing periods in the time series, we can start handling missing values by filling as many as possible. Firstly, count the number of missing values in each column:

In [10]:
df.isnull().sum()

timeStamp    2  
demand       83 
precip       232
temp         188
dtype: int64

Missing timeStamp can be filled from the dataframe index:



In [11]:
df.loc[df.isnull().all(axis=1), 'timeStamp'] = df.loc[df.isnull().all(axis=1)].index

For the other columns, we can fill many missing values by interpolating between the two closest non-missing values. Here, we use a quadratic function and set a limit of 6. This limit means that if more than 6 missing values occur consecutively, the missing values are not interpolated over and they remain missing. This is to avoid spurious interpolation between very distant time periods.

In [12]:
df = df.interpolate(limit=6, method='linear')

Fill missing precip values with common value of 0:

In [13]:
precip_mode = np.asscalar(stats.mode(df['precip']).mode)
df['precip'] = df['precip'].fillna(precip_mode)

In [14]:
df.isnull().sum()

timeStamp    0 
demand       43
precip       0 
temp         86
dtype: int64

The number of missing values has now been greatly reduced. Records containing the remaining missing values will be removed later after model features have been created.

## Compute features for forecasting models
After exploring the data, it is clear that the energy demand follows seasonal trends, with daily, weekly and annual periodicity. We will create features that encode this information. First, we compute time driven features based on timeStamp. Note for dayofweek, Monday=0 and Sunday=6.

In [15]:
df_features = df.copy()

In [16]:
df_features['hour'] = df_features.timeStamp.dt.hour
df_features['month'] = df_features.timeStamp.dt.month-1
df_features['dayofweek'] = df_features.timeStamp.dt.dayofweek

Compute lagged demand features

In [17]:
def generate_lagged_features(df, var, max_lag):
    for t in range(1, max_lag+1):
        df[var+'_lag'+str(t)] = df[var].shift(t, freq='1H')

In [18]:
generate_lagged_features(df_features, 'temp', 6)
generate_lagged_features(df_features, 'demand', 6)

In [19]:
df_features.head()

,timeStamp,demand,precip,temp,hour,month,dayofweek,temp_lag1,temp_lag2,temp_lag3,temp_lag4,temp_lag5,temp_lag6,demand_lag1,demand_lag2,demand_lag3,demand_lag4,demand_lag5,demand_lag6
2012-01-01 00:00:00,2012-01-01 00:00:00,4937.5,0.0,46.13,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 01:00:00,2012-01-01 01:00:00,4752.1,0.0,45.89,1,0,6,46.13,NaN,NaN,NaN,NaN,NaN,4937.5,NaN,NaN,NaN,NaN,NaN
2012-01-01 02:00:00,2012-01-01 02:00:00,4542.6,0.0,45.04,2,0,6,45.89,46.13,NaN,NaN,NaN,NaN,4752.1,4937.5,NaN,NaN,NaN,NaN
2012-01-01 03:00:00,2012-01-01 03:00:00,4357.7,0.0,45.03,3,0,6,45.04,45.89,46.13,NaN,NaN,NaN,4542.6,4752.1,4937.5,NaN,NaN,NaN
2012-01-01 04:00:00,2012-01-01 04:00:00,4275.5,0.0,42.61,4,0,6,45.03,45.04,45.89,46.13,NaN,NaN,4357.7,4542.6,4752.1,4937.5,NaN,NaN


## Final data cleaning and write out training and test datasets
Count remaining null values.

In [20]:
df_features.isnull().sum()

timeStamp      0 
demand         43
precip         0 
temp           86
hour           0 
month          0 
dayofweek      0 
temp_lag1      87
temp_lag2      88
temp_lag3      89
temp_lag4      90
temp_lag5      91
temp_lag6      92
demand_lag1    43
demand_lag2    43
demand_lag3    43
demand_lag4    43
demand_lag5    43
demand_lag6    43
dtype: int64

Count number of rows with any null values

In [21]:
df_features.loc[df_features.isnull().any(axis=1), ].shape[0]

153

This is a very small proportion of the overall dataset so can be safely dropped.

In [22]:
df_features.dropna(how='any', inplace=True)

Split data into training and test datasets. All data after 1st July 2016 is reserved for the test set.

In [23]:
train, test = (df_features.loc[df_features['timeStamp']<'2016-07-01'], df_features.loc[df_features['timeStamp']>='2016-07-01'])

## Instantiate Auto ML Regressor

Instantiate a AutoML Object This creates an Experiment in Azure ML. You can reuse this objects to trigger multiple runs. Each run will be part of the same experiment.

|Property|Description|
|-|-|
|**primary_metric**|This is the metric that you want to optimize.<br> Auto ML Regressor supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i>|
|**max_time_sec**|Time limit in seconds for each iterations|
|**iterations**|Number of iterations. In each iteration Auto ML Classifier trains the data with a specific pipeline|
|**n_cross_validations**|Number of cross validation splits|

In [24]:
from azureml.train.automl import AutoMLRegressor
import time
import logging

automl_regressor = AutoMLRegressor(project = project,
                                   name = "AutoML_Demo_Experiment_v3",
                                   max_time_sec = 600,
                                   iterations = 10,
                                   primary_metric = 'normalized_root_mean_squared_error', 
                                   n_cross_validations = 5,
                                   debug_log = 'automl.log',
                                   verbosity = logging.INFO)

In [25]:
X = train.drop(['demand', 'timeStamp'], axis=1)
X.head()
type(X)
y = train['demand']
y = y.values
type(y)
#X = X.values
type(y)

numpy.ndarray

## Training the Model

You can call the fit method on the AutoML instance and pass the run configuration. For Local runs the execution is synchronous. Depending on the data and number of iterations this can run for while.
You will see the currently running iterations printing to the console.

*fit* method on Auto ML Regressor triggers the training of the model. It can be called with the following parameters

|**Parameter**|**Description**|
|-|-|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification.|
|**compute_target**|Indicates the compute used for training. <i>local</i> indicates train on the same compute which hosts the jupyter notebook. <br>For DSVM and Batch AI please refer to the relevant notebooks.|
|**show_output**| True/False to turn on/off console output|

In [26]:
local_run = automl_regressor.fit( X = X, 
                                  y = y, 
                                  compute_target = 'local', 
                                  show_output = True ) 

Running locally
Parent Run ID: AutoML_eeecbb34-59f9-4e58-a757-6897e2d6666e
***********************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE:  A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
***********************************************************************************************

 ITERATION     PIPELINE                               DURATION                METRIC      BEST
         0      Normalize RF regressor                0:00:09.988100           0.050     0.050
         1      Normalize lightGBM regressor          0:00:14.730993           0.010     0.010
         2      Robust Scaler extra trees regressor   0:00:20.434624           0.021     0.010
         3      Scale 0/1 Lasso lars                  0:00:08.538191           0.010     0.010

## Exploring the results

#### Widget for monitoring runs

The widget will sit on "loading" until the first iteration completed, then you will see an auto-updating graph and table show up. It refreshed once per minute, so you should see the graph update as child runs complete.

NOTE: The widget displays a link at the bottom. This links to a web-ui to explore the individual run details.

In [27]:
from azureml.train.widgets import RunDetails
RunDetails(local_run).show() 

AutoML(widget_settings={'childWidgetDisplay': 'popup'})


#### Retrieve All Child Runs
You can also use sdk methods to fetch all the child runs and see individual metrics that we log. 

In [28]:
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if not isinstance(v, list)}    
    metricslist[int(properties['iteration'])] = metrics
    
import pandas as pd
import seaborn as sns
rundata = pd.DataFrame(metricslist).sort_index(1)
cm = sns.light_palette("lightgreen", as_cmap = True)
s = rundata.style.background_gradient(cmap = cm)
s

,0,1,2,3,4,5,6,7,8,9
explained_variance,0.883569,0.994874,0.979624,0.994953,0.99672,0.998458,0.0882075,0.914448,0.978354,0.99693
mean_absolute_error,336.036,56.7647,120.516,64.2049,50.2717,33.2532,918.14,256.774,131.19,181.768
median_absolute_error,283.912,38.2161,80.9027,44.5879,35.7801,22.9815,738.36,175.634,94.0057,172.511
normalized_mean_absolute_error,0.0390904,0.00660332,0.0140194,0.00746881,0.00584799,0.00386827,0.106805,0.0298699,0.015261,0.0211446
normalized_median_absolute_error,0.0330268,0.00444559,0.00941123,0.00518681,0.00416222,0.00267339,0.0858918,0.0204311,0.0109355,0.0200678
normalized_root_mean_squared_error,0.0497262,0.0104345,0.0208041,0.010356,0.00834678,0.00572173,0.139203,0.042592,0.0214437,0.0225197
normalized_root_mean_squared_error_min,0.0497262,0.0104345,0.0104345,0.010356,0.00834678,0.00572173,0.00572173,0.00572173,0.00572173,0.00572173
normalized_root_mean_squared_log_error,8.07741e-06,1.62331e-06,3.27666e-06,1.7441e-06,1.34689e-06,9.06972e-07,2.25201e-05,6.57259e-06,3.479e-06,3.68513e-06
r2_score,0.883548,0.994873,0.979624,0.994952,0.99672,0.998458,0.0880561,0.914441,0.978353,0.976128
root_mean_squared_error,427.466,89.699,178.841,89.0245,71.7523,49.1863,1196.64,366.138,184.338,193.589


### Retrieve the Best Model

Below we select the best pipeline from our iterations. The *get_output* method on automl_classifier returns the best run and the fitted model for the last *fit* invocation. There are overloads on *get_output* that allow you to retrieve the best run and fitted model for *any* logged metric or a particular *iteration*.

In [29]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run({'id': 'AutoML_eeecbb34-59f9-4e58-a757-6897e2d6666e_5', 'type': None, 'status': 'Completed'})
Pipeline(memory=None,
     steps=[('Normalize', StandardScaler(copy=True, with_mean=False, with_std=True)), ('lightGBM regressor', <azureml.train.automl.model_wrappers.LightGBMRegressor object at 0x000001AE33FA3438>)])


#### Best Model based on any other metric
Show the run and model that has the smallest `spearman_correlation` value:

In [37]:
lookup_metric = "spearman_correlation"
best_run, fitted_model = local_run.get_output(metric=lookup_metric)
print(best_run)
print(fitted_model)

Run({'id': 'AutoML_eeecbb34-59f9-4e58-a757-6897e2d6666e_5', 'type': None, 'status': 'Completed'})
Pipeline(memory=None,
     steps=[('Normalize', StandardScaler(copy=True, with_mean=False, with_std=True)), ('lightGBM regressor', <azureml.train.automl.model_wrappers.LightGBMRegressor object at 0x000001AE340D43C8>)])


#### Best Model based on any iteration
Simply show the run and model from the 3rd iteration:

In [38]:
iteration = 3
third_run, third_model = local_run.get_output(iteration = iteration)
print(third_run)
print(third_model)

Run({'id': 'AutoML_eeecbb34-59f9-4e58-a757-6897e2d6666e_3', 'type': None, 'status': 'Completed'})
Pipeline(memory=None,
     steps=[('Scale 0/1', MinMaxScaler(copy=True, feature_range=(0, 1))), ('Lasso lars', LassoLars(alpha=0.001, copy_X=True, eps=2.220446049250313e-16,
     fit_intercept=True, fit_path=True, max_iter=500, normalize=True,
     positive=False, precompute='auto', verbose=False))])


### Register fitted model for deployment

In [39]:
description = 'AutoML Model'
tags = None
local_run.register_model(description = description, tags = tags)
print(local_run.model_id) # Use this id to deploy the model as a web service in Azure

Registering model AutoMLeeecbb345best
AutoMLeeecbb345best


### Testing the Fitted Model

Predict on training and test set, and calculate residual values.

In [40]:
x_train = train.drop(['demand', 'timeStamp'], axis=1)
y_train = train['demand']
y_pred_train = fitted_model.predict(x_train)
#y_residual_train = y_train - y_pred_train

x_test = test.drop(['demand', 'timeStamp'], axis=1)
y_test = test['demand']
y_pred_test = fitted_model.predict(x_test)
#y_residual_test = y_test - y_pred_test


expected = y_train
predictions = y_pred_train
forecast_errors = [expected[i]-predictions[i] for i in range(len(expected))]
#print('Forecast Errors: %s' % forecast_errors)